<a href="https://colab.research.google.com/github/javialra97/IQ/blob/main/cp_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clase Práctica #7 de Informática Química

Es esta clase práctica trabajaremos directamente en la nube, haciendo uso de las facilidades ofrecidas por Google a través de **Google Colaboratory**.

El objetivo es que a través del lenguaje de programación Python obtendramos algunas propiedades químicas como pueden ser las cargas parciales, momento dipolo y geometría de la molecula de agua.

Para esto necesitamos una biblioteca que permita realizar este tipo de operaciones. Para eso usaremos el programa PySCF{cita}. Este programa esta escrito totalmente en Python de manera que uno pueda importar las bibliotecas y funciones que necesite. Para más información puede tocar [aquí](https://pyscf.org/)

Como es de esperar estas funciones no forman parte de las bibliotecas estándar de Python por lo que lo primero que hace falta es instalar. En Python hay varias maneras de instalar este tipo de módulos, no ahondaremos en eso, sencillamente escogeremos uno de ellos, [pip](https://pypi.org/project/pip/). 


In [ ]:
! pip install pyscf
! pip install pyberny
! pip install mogli

Las dos primeras líneas instalan el programa PySCF. Mogli nos permitira visualizar la molécula de agua desde el mismo navegador. Hay muchos programas que permiten esto (GaussView, Avogadro, ChemCraft ...) pero el sentido es hacerlo todo con códigos de Python. La informacion de Mogli la pueden encontrar en su [repositorio de GitHub](https://github.com/sciapp/mogli).

Una vez que estén instalados las bibliotecas necesarias, cual sería el próximo paso🤔? ... Tendremos que importar aquellas funciones que nos permitirán realizar el cálculos de las propiedades de interés.

In [ ]:
from pyscf import gto, scf
from pyscf.geomopt.berny_solver import optimize

La primera línea esta importando aquellas funciones que nos permitirán resolver de manera aproximada una de las más famosas ecuaciones de la mecánica cuántica, la ecuación de Schröndinger:

HY = EY (poner en LaTex ...)

Como nuestro interés es encontrar la geometría de equilibrio, tendremos que buscar una función que optimice la estructura inicial que le daremos como entrada. Extrapolando un poco los conocimientos adquiridos en Análisis Matemático I, lo que estamos buscando es el conjunto óptimos de parámetros que describan a la molécula de agua, estos serían la distancia de enlace O-H y el ángulo de enlace HOH.

La entrada de nuestro programa queda claro que es la molécula de agua pero hasta el momento no la hemos definido. Necesitamos escribirla de una manera que nuestro programa entienda que es la molécula de agua (un poco abstracto ehh..). Hay varias maneras de hacer esto, nosotros escogeremos la representación cartesiana. Tendriamos que declarar las coordenadas xyz de tres puntos en el espacio. 

In [ ]:
mol_agua = gto.M (atom = ''' O 0 0 0; H 0 1 0; H 0 0 1 ''', basis = 'sto-3g',)

En la primera linea declaramos a nuestra molécula de agua. Fijense que el átomo de oxígeno se encuentra en el centro del eje de coordenadas mientras que los átomos de hidrógeno se encuentran desplazados una unidad en el eje de las *y* y en el eje de las *z*. 

Ahora vamos a buscar el mínimo(optimizar) y mostrar las coordenas de esta nueva geometría.

In [ ]:
mol_eq = optimize(mol_agua)
print("\n")
print(mol_eq.atom_coords())

Como la solución de este sistema es por vía numérica, verán que aparecen diferentes iteraciones y al final podemos ver como los átomos se desplazaron de sus posiciones iniciales. Un pequeño calculo de distancia euclidiana nos muestra como las distancias son diferentes de la unidad.



In [ ]:
import numpy as np
coord_O = np.array([mol_eq.atom_coords()[0][0], mol_eq.atom_coords()[0][1], mol_eq.atom_coords()[0][2]])
coord_H = np.array([mol_eq.atom_coords()[1][0], mol_eq.atom_coords()[1][1], mol_eq.atom_coords()[1][2]])
dist_OH = np.linalg.norm(coord_O - coord_H)
print(dist_OH)

La distancia anterior esta en Bohr, que es aproximadamente 1 A, la cual es la que se reporta para la molécula de agua, recuerden que la molécula inicial tenia una distancia de enlace de 1 Bohr.

Ahora vamos a calcular algunas propiedades de interés a una molécula de agua en fase gaseosa.


In [ ]:
mol_eq.verbose = 5
hf_calc = mol_eq.RHF()
hf_calc.kernel()
hf_calc.analyze()

En estas líneas se esta resolviendo de manera aproximada la ecuación de Schrödinger. La primera línea específica cuan detallada es la salida.

En el principio de la salida podemos ver los paramétros estándar del programa puesto que nosotros no hemos ajustado ninguno, solo que el método sea RHF(*Restricted Hartree-Fock*)

Como les habiamos comentado anteriormente, esta solución se basa en un método iterativo, por eso ven varios ciclos con resultados similares y en cada uno muestra el tiempo de ejecución en CPU. Recuerden que este entorno nos permite hacer uso de GPU y TPU pero en este caso no es necesario reservar ese tipo de recurso.

Una vez que terminen los ciclos nos muestran un resumen con los principales valores energéticos así como las energías y coeficientes de los Orbitales Moleculares (no se asusten, este término lo dominarán a partir del próximo semestre)

En el final se pueden ver las cargas parciales atómicas (de acuerdo al esquema de partición de Mulliken) así como la descomposición del momento dipolo en cada eje.

Una vez terminada esta parte vamos ahora a visualizar esta molécula usando la biblioteca Mogli.

Para esto necesitamos crear un fichero con extensión *.xyz*. 


In [ ]:
coor = mol_eq.atom_coords()
label = ['O', 'H', 'H']
print(coor)

with open('agua_eq.xyz', 'w') as file:
  file.write('3\n\n')
  for i,j in zip(label, coor):
    file.write(i + '  ' + str(j[0]) + '  ' + str(j[1]) + '  ' + str(j[2]) + ' \n')

De esta manera ya creamos el fichero de coordenadas de la molecula de agua optimizada en formato *.xyz*. A la izquierda podemos ver un simbolo de una carpeta, den click ahi y veran que ahora hay un fichero que se llama agua_eq.xyz. Den doble click y cuando se les descargue mirenlo con un editor de texto

Una vez que ya tenemos preparado la entrada para las funciones de la biblioteca Mogli vamos a importarlas y generar la visualizacion de dos maneras distintas.

In [ ]:
import mogli, gr
molecule = mogli.read('agua_eq.xyz')
mogli.bond_method = 'constant_delta'
mogli.BOND_RADIUS = 0.50
mogli.export(molecule[0], 'agua_eq.html', camera=((40,0,0),(0,0,0),(0,1,0)), bonds_param= 1.95)